In [7]:
from pyomo.environ import *
from pyomo.opt import SolverFactory

In [8]:
m = ConcreteModel()

m.Chi2Bos = Var(domain=NonNegativeReals)
m.Chi2New = Var(domain=NonNegativeReals)
m.Chi2Tor = Var(domain=NonNegativeReals)
m.Det2Bos = Var(domain=NonNegativeReals)
m.Det2New = Var(domain=NonNegativeReals)
m.Det2Tor = Var(domain=NonNegativeReals)

m.Boston_Demand = Constraint(expr = m.Chi2Bos + m.Det2Bos >= 0.5)
m.Newark_Demand = Constraint(expr = m.Chi2New + m.Det2New >= 1.5)
m.Toronto_Demand = Constraint(expr = m.Chi2Tor + m.Det2Tor >= 1)
m.Toronto_Supply = Constraint(expr = m.Chi2Bos + m.Chi2New + m.Chi2Tor <= 2)
m.Detrit_Supply = Constraint(expr = m.Det2Bos + m.Det2New + m.Det2Tor <= 1.5)

m.TotalCost = Objective(expr = 4*m.Chi2Bos + 2.5*m.Chi2New + 1.5*m.Chi2Tor + 
                        3*m.Det2Bos + 1.5*m.Det2New + m.Det2Tor, 
                        sense=minimize)

opt = SolverFactory('glpk')
sol = opt.solve(m) 
print(m.Chi2Bos(), m.Chi2New(), m.Chi2Tor(), 
      m.Det2Bos(), m.Det2New(), m.Det2Tor(),  m.TotalCost())

0.0 0.5 1.0 0.5 1.0 0.0 5.75


In [10]:
m.display()

Model unknown

  Variables:
    Chi2Bos : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : False : False : NonNegativeReals
    Chi2New : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.5 :  None : False : False : NonNegativeReals
    Chi2Tor : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   1.0 :  None : False : False : NonNegativeReals
    Det2Bos : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.5 :  None : False : False : NonNegativeReals
    Det2New : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   1.0 :  None : False : False : NonNegativeReals
    Det2Tor : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   0.0 :  None : Fal

In [11]:
m = ConcreteModel()

m.ORIGINS = Set(initialize=['Chi','Det']) 
m.DEST = Set(initialize=['Bos','New','Tor']) 

m.dem = Param(m.DEST, initialize={'Bos':0.5, 'New':1.5, 'Tor':1})
m.avail = Param(m.ORIGINS, initialize={'Chi':2, 'Det':1.5})
cost = {('Chi', 'Bos'): 4,
        ('Chi', 'New'): 2.5,
        ('Chi', 'Tor'): 1.5,
        ('Det', 'Bos'): 3,
        ('Det', 'New'): 1.5,
        ('Det', 'Tor'): 1}
m.cost = Param(m.ORIGINS, m.DEST, initialize=cost)

m.Pounds = Var(m.ORIGINS,m.DEST,domain=NonNegativeReals)

def Demand_Rule(m,d):
    return sum(m.Pounds[o,d] for o in m.ORIGINS) >= m.dem[d]
m.Demand = Constraint(m.DEST, rule=Demand_Rule)

def Supply_Rule(m,o):
    return sum(m.Pounds[o,d] for d in m.DEST) <= m.avail[o]
m.Supply = Constraint(m.ORIGINS, rule=Supply_Rule)

def TotalCost_Rule(m):
    return (sum(m.cost[o,d]*m.Pounds[o,d] for o in m.ORIGINS for d in m.DEST))
m.TotalCost = Objective(rule=TotalCost_Rule, sense=minimize)

opt = SolverFactory('glpk')
sol = opt.solve(m) 
print([value(m.Pounds[o,d]) for o in m.ORIGINS for d in m.DEST], m.TotalCost())


[0.0, 0.5, 1.0, 0.5, 1.0, 0.0] 5.75


In [12]:
m.display()

Model unknown

  Variables:
    Pounds : Size=6, Index=ORIGINS*DEST
        Key            : Lower : Value : Upper : Fixed : Stale : Domain
        ('Chi', 'Bos') :     0 :   0.0 :  None : False : False : NonNegativeReals
        ('Chi', 'New') :     0 :   0.5 :  None : False : False : NonNegativeReals
        ('Chi', 'Tor') :     0 :   1.0 :  None : False : False : NonNegativeReals
        ('Det', 'Bos') :     0 :   0.5 :  None : False : False : NonNegativeReals
        ('Det', 'New') :     0 :   1.0 :  None : False : False : NonNegativeReals
        ('Det', 'Tor') :     0 :   0.0 :  None : False : False : NonNegativeReals

  Objectives:
    TotalCost : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True :  5.75

  Constraints:
    Demand : Size=3
        Key : Lower : Body : Upper
        Bos :   0.5 :  0.5 :  None
        New :   1.5 :  1.5 :  None
        Tor :   1.0 :  1.0 :  None
    Supply : Size=2
        Key : Lower : Body : Upper
        Chi : 